Makes summary df from abcTau run results.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns 

import pickle
import re
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import gaussian_kde

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\AppData\\Local\\anaconda3\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau

from isttc.scripts.cfg_global import project_folder_path

In [2]:
def compute_map_1d_abctau(result):
    """
    Compute the 1D MAP estimate from abcTau output.
    Method as in Zeraati et al. (2022):
      1. Smooth the posterior with Gaussian KDE
      2. Locate the maximum via grid search (take into account weights)
    """
    # Extract accepted parameter samples and weights
    theta = np.asarray(result['theta accepted']).squeeze()
    weights = np.asarray(result['weights'])
    weights = weights / np.sum(weights)  # normalize to 1

    # Fit weighted Gaussian KDE (posterior smoothing)
    kde = gaussian_kde(theta, weights=weights)

    # Create fine grid over parameter space
    grid = np.linspace(theta.min(), theta.max(), 2000)

    # Evaluate density and find peak
    density = kde(grid)
    theta_map = grid[np.argmax(density)]

    return theta_map, grid, density

In [14]:
dataset_folder = project_folder_path + 'synthetic_dataset\\'
results_folder = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau\\'

results_folder_abctau = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau_abctau\\'
#datasave_path = results_folder_abctau + 'all_abctau\\'
datasave_path = results_folder_abctau + 'all_abctau_dst_gamma\\final_results'

### Load data (spike trains and calculated acf's)

In [12]:
df_all = pd.read_pickle(results_folder + "summary_tau_all_long_df_all_units.pkl")
df_all.head()

,unit_id,tau,tau_lower,tau_upper,fit_r_squared,acf_decline,method,tau_ms,fr,alpha,tau_ms_true,tau_diff_abs,tau_diff_rel,ci_width,lv
0,0,2.217899,2.119224,2.316575,0.998369,True,acf_full,110.894975,7.741821,0.851099,150.923515,40.028540,26.522401,0.197350,1.307775
1,1,2.568210,2.437150,2.699270,0.997899,True,acf_full,128.410512,4.394396,0.858098,147.947233,19.536721,13.205195,0.262120,1.345006
2,2,5.227451,4.334421,6.120482,0.984960,True,acf_full,261.372568,8.587393,0.528805,267.002290,5.629722,2.108492,1.786061,1.068416
3,3,3.950209,2.250084,5.650335,0.889719,True,acf_full,197.510466,6.976707,0.300692,194.403797,3.106670,1.598050,3.400250,1.055631
4,4,3.717963,3.426456,4.009471,0.995737,True,acf_full,185.898174,0.950832,0.691458,239.986489,54.088315,22.538067,0.583014,1.357205


### Summary df

In [15]:
directory = Path(datasave_path)
files = [file for file in directory.iterdir() if file.is_file()]
print(f'N files {len(files)}')

records = []
for file in files:
    # print(file.name)  
    # the last iteration is encoded in the name
    ind = file.name.find('steps') 
    final_step = int(file.name[ind+5] + file.name[ind+6])
    # het results 
    abc_results = np.load(file, allow_pickle=True)
    theta_accepted = abc_results[final_step-1]['theta accepted']
    tau = theta_accepted[0]
    theta_map, grid, density = compute_map_1d_abctau(abc_results[final_step-1])

    match_unit_id = re.search(r"spike_train_(\d+)_", file.name)
    unit_id = int(match_unit_id.group(1))
    records.append({
        'unit_id': unit_id,
        'tau_mean': np.mean(tau),
        'tau_median': np.median(tau),
        'tau_map': theta_map,
        'method': 'abctau_full',
    })

tau_df = pd.DataFrame(records)

N files 33


In [16]:
tau_df.head()

,unit_id,tau_mean,tau_median,tau_map,method
0,0,221.774606,186.126551,105.512055,abctau_full
1,1,281.902754,253.817253,204.615640,abctau_full
2,2,287.216948,269.312221,166.563892,abctau_full
3,3,188.307203,163.736487,60.134810,abctau_full
4,4,235.535417,228.100416,203.376773,abctau_full


In [17]:
tau_df_merged = tau_df.merge(df_all[df_all['method'] == "acf_full"][['unit_id', 'fr', 'alpha', 'tau_ms_true', 'lv']], 
                             how='left', on='unit_id')
tau_df_merged['tau_diff_abs'] = np.abs(tau_df_merged['tau_map'] - tau_df_merged['tau_ms_true'])
tau_df_merged['tau_diff_rel'] = tau_df_merged['tau_diff_abs'] / tau_df_merged['tau_ms_true'] * 100
tau_df_merged.sort_values(by='tau_diff_rel')

tau_df_merged.rename(columns={'tau_map': 'tau_ms'}, inplace=True)
tau_df_merged.drop('tau_median', axis=1, inplace=True)
tau_df_merged.drop('tau_mean', axis=1, inplace=True)

tau_df_merged

,unit_id,tau_ms,method,fr,alpha,tau_ms_true,lv,tau_diff_abs,tau_diff_rel
0,0,105.512055,abctau_full,7.741821,0.851099,150.923515,1.307775,45.411460,30.089055
1,1,204.615640,abctau_full,4.394396,0.858098,147.947233,1.345006,56.668408,38.303121
2,2,166.563892,abctau_full,8.587393,0.528805,267.002290,1.068416,100.438397,37.617055
3,3,60.134810,abctau_full,6.976707,0.300692,194.403797,1.055631,134.268986,69.067060
4,4,203.376773,abctau_full,0.950832,0.691458,239.986489,1.357205,36.609716,15.254907
5,5,171.866066,abctau_full,9.756467,0.629259,147.666392,1.168037,24.199674,16.388072
6,6,77.775251,abctau_full,7.613786,0.800260,112.904124,1.317774,35.128873,31.113898
7,7,333.682611,abctau_full,7.862782,0.781671,229.046160,1.246275,104.636451,45.683565
8,8,117.028320,abctau_full,1.289855,0.322682,166.766873,1.381250,49.738553,29.825200
9,9,235.276668,abctau_full,4.509356,0.567198,206.615257,1.231979,28.661411,13.871875


In [18]:
df_all_subset = df_all[df_all['unit_id'].isin(tau_df['unit_id'].values)][['unit_id', 'method', 'tau_ms', 'fr', 'alpha', 'tau_ms_true',
       'tau_diff_abs', 'tau_diff_rel', 'lv']].copy()
df_all_subset.reset_index(drop=True, inplace=True)
df_all_subset

,unit_id,method,tau_ms,fr,alpha,tau_ms_true,tau_diff_abs,tau_diff_rel,lv
0,0,acf_full,110.894975,7.741821,0.851099,150.923515,40.028540,26.522401,1.307775
1,1,acf_full,128.410512,4.394396,0.858098,147.947233,19.536721,13.205195,1.345006
2,2,acf_full,261.372568,8.587393,0.528805,267.002290,5.629722,2.108492,1.068416
3,3,acf_full,197.510466,6.976707,0.300692,194.403797,3.106670,1.598050,1.055631
4,4,acf_full,185.898174,0.950832,0.691458,239.986489,54.088315,22.538067,1.357205
...,...,...,...,...,...,...,...,...,...
127,28,sttc_trial_concat,95.994731,1.551352,0.869480,290.790300,194.795568,66.988331,1.360630
128,29,sttc_trial_concat,120.327003,6.833659,0.195178,180.314131,59.987128,33.268124,1.027506
129,30,sttc_trial_concat,88.978593,7.450174,0.412868,150.464001,61.485408,40.863866,1.123272
130,31,sttc_trial_concat,5.009485,9.675422,0.197857,150.122967,145.113482,96.663079,1.023090


In [19]:
df_5_methods = pd.concat([tau_df_merged, df_all_subset])
df_5_methods.reset_index(drop=True, inplace=True)

df_5_methods

,unit_id,tau_ms,method,fr,alpha,tau_ms_true,lv,tau_diff_abs,tau_diff_rel
0,0,105.512055,abctau_full,7.741821,0.851099,150.923515,1.307775,45.411460,30.089055
1,1,204.615640,abctau_full,4.394396,0.858098,147.947233,1.345006,56.668408,38.303121
2,2,166.563892,abctau_full,8.587393,0.528805,267.002290,1.068416,100.438397,37.617055
3,3,60.134810,abctau_full,6.976707,0.300692,194.403797,1.055631,134.268986,69.067060
4,4,203.376773,abctau_full,0.950832,0.691458,239.986489,1.357205,36.609716,15.254907
...,...,...,...,...,...,...,...,...,...
160,28,95.994731,sttc_trial_concat,1.551352,0.869480,290.790300,1.360630,194.795568,66.988331
161,29,120.327003,sttc_trial_concat,6.833659,0.195178,180.314131,1.027506,59.987128,33.268124
162,30,88.978593,sttc_trial_concat,7.450174,0.412868,150.464001,1.123272,61.485408,40.863866
163,31,5.009485,sttc_trial_concat,9.675422,0.197857,150.122967,1.023090,145.113482,96.663079


In [20]:
df_5_methods.to_pickle(results_folder + 'summary_tau_all_long_df_all_units_with_abctau_dst_gamma_0_01_50.pkl')